## 1. Import and Install Dependencies

In [2]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import time
import mediapipe as mp
from tensorflow.keras.models import load_model

## 2. Keypoints using MP Holistic

In [3]:
mp_holistic = mp.solutions.holistic # holistic model
mp_drawing = mp.solutions.drawing_utils # drawing utilities

In [4]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    
    return image, results

In [5]:
def draw_landmarks(images, results):
    mp_drawing.draw_landmarks(images, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(images, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [6]:
def draw_styled_landmarks(images, results):
    
    # Draw left hand connections
    mp_drawing.draw_landmarks(images, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    
    # Draw right hand connections    
    mp_drawing.draw_landmarks(images, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) 
    

In [6]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=.5, min_tracking_confidence=.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # Flip images
        image = cv2.flip(image, 1)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q' or 'Q'):
            break

cap.release()
cv2.destroyAllWindows()

## 3. Extract Keypoint Values

In [7]:
def hand_extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

## 4. Setup Folders for Collection

In [8]:
# path ofr exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data_test')

# Actions that we try to detect
actions = np.array(['nothing', 'add_count', 'reset_count'])

# Thirty videos worth of data
no_sequences = 40

# Video are going to be 30 frames in length
sequence_length = 20

In [9]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

## 5. Collect Keypoint Values for Training and Testing

In [1]:
cap = cv2.VideoCapture(0)
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=.5, min_tracking_confidence=.5) as holistic:
    
    # New Loop
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video lenght aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # Apply wait logic
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {0} Video Number {1}'.format(action, sequence), (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'Collecting frames for {0} Video Number {1}'.format(action, sequence), (15, 12),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = hand_extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q' or 'Q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [ ]:
cap.release()
cv2.destroyAllWindows()

## 6. Preprocess Data and Create Labels and Features

In [10]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [11]:
label_map = {label:num for num, label in enumerate(actions)}

In [12]:
label_map

{'nothing': 0, 'add_count': 1, 'reset_count': 2}

In [13]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), '{}.npy'.format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [ ]:
np.array(sequences).shape

(120, 20, 126)

In [ ]:
np.array(sequences).shape

(120, 20, 126)

In [ ]:
np.array(labels).shape

(120,)

In [ ]:
x = np.array(sequences)

In [ ]:
x.shape

(120, 20, 126)

In [19]:
y = to_categorical(labels).astype(int)

In [20]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=.1)

In [21]:
x_test.shape, y_test.shape

((12, 20, 126), (12, 3))

## 7. Build and Train LSTM Neural Networkto_categorical

In [20]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Input
from tensorflow.keras.callbacks import TensorBoard

In [21]:
log_dir = os.path.join('Logs_RNN')
tb_callback = TensorBoard(log_dir=log_dir)

In [23]:
input_shape = Input(shape=(20, 126))
layer = LSTM(64, return_sequences=True, activation='relu')(input_shape)
layer = LSTM(128, return_sequences=True, activation='relu')(layer)
layer = LSTM(64, return_sequences=False, activation='relu')(layer)
layer = Dense(64, activation='relu')(layer)
layer = Dense(32, activation='relu')(layer)
output = Dense(actions.shape[0], activation='softmax')(layer)

In [22]:
input_shape = Input(shape=(20, 126))
layer = SimpleRNN(64, return_sequences=True, activation='relu')(input_shape)
layer = SimpleRNN(128, return_sequences=True, activation='relu')(layer)
layer = SimpleRNN(64, return_sequences=False, activation='relu')(layer)
layer = Dense(64, activation='relu')(layer)
layer = Dense(32, activation='relu')(layer)
output = Dense(actions.shape[0], activation='softmax')(layer)

In [23]:
# model = Model(input_shape, output, name='LSTM')
model = Model(input_shape, output, name='RNN')

In [24]:
model.summary()

Model: "RNN"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 20, 126)]         0         
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 20, 64)            12224     
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 20, 128)           24704     
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 64)                12352     
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99      

In [25]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [26]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint('best-model_RNN.h5')
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="loss", patience=3, restore_best_weights=True)

In [27]:
model.fit(x_train, y_train, epochs=30, callbacks=[tb_callback, checkpoint_cb])

Epoch 1/30
1/4 [======>.......................] - ETA: 0s - loss: 1.1353 - categorical_accuracy: 0.4375WARNING:tensorflow:From C:\Users\hoya9\anaconda3\envs\tf-gpu\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
4/4 [==============================] - 0s 78ms/step - loss: 1.1092 - categorical_accuracy: 0.3889
Epoch 2/30
4/4 [==============================] - 0s 40ms/step - loss: 0.9915 - categorical_accuracy: 0.5278
Epoch 3/30
4/4 [==============================] - 0s 49ms/step - loss: 0.8561 - categorical_accuracy: 0.7778
Epoch 4/30
4/4 [==============================] - 0s 43ms/step - loss: 0.7051 - categorical_accuracy: 0.8056
Epoch 5/30
4/4 [==============================] - 0s 36ms/step - loss: 0.4692 - categorical_accuracy: 0.8796
Epoch 6/30
4/4 [==============================] - 0s 32ms/step - loss

## 9. Load Weights

In [22]:
# model = load_model('C:/Users/hoya9/Desktop/CV/MediaPipe/best-model_LSTM.h5')
model = load_model('C:/Users/hoya9/Desktop/CV/MediaPipe/best-model_RNN.h5')

## 8. Make Predictions

In [23]:
res = model.predict(x_test)

In [24]:
actions[np.argmax(res[4])]

'add_count'

In [25]:
actions[np.argmax(y_test[4])]

'add_count'

## 10. Evaluation using Confusion Matrix and Accuracy

In [31]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [32]:
yhat = model.predict(x_test)

In [33]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [34]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[9, 0],
        [0, 3]],

       [[7, 0],
        [0, 5]],

       [[8, 0],
        [0, 4]]], dtype=int64)

In [35]:
accuracy_score(ytrue, yhat)

1.0

## 11. Test Real Time

In [33]:
colors = [(245, 117, 16), (117, 245, 16), (16, 117, 245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0, 60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [32]:
sequence = []
sentence = []
predictions = []
threshold = 0.7

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = hand_extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-20:]
        
        if len(sequence) == 20:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            predictions.append(np.argmax(res))
            
            
        #3. Viz logic
            if np.unique(predictions[-10:])[0]==np.argmax(res): 
                if res[np.argmax(res)] > threshold: 
                    
                    if len(sentence) > 0: 
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                    else:
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 3: 
                sentence = sentence[-3:]
                
            image = cv2.flip(image,1)

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
        
        cv2.rectangle(image, (0,0), (640, 40), (0, 0, 0), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [33]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
x_test[0].shape

In [ ]:
np.expand_dims(x_test[0], axis=0).shape

In [ ]:
res[np.argmax(res)] > threshold

In [ ]:
model.predict(np.expand_dims(x_test[0], axis=0))